# Performance Evaluation: Artha Optimum vs Benchmark portfolio

The following project compares the returns and risk levels of holding the fund Artha Optimum (a medium risk-profile fund listed on OMX Copenhagen) against a comparable portfolio. The portfolio consists of 50 % in Nykredits mortgage bond index (a well known benchmark for danish bond mortgages), and 50 % in MSCIs World index ( a well known benchmark for global stocks). 

> **Note the following non-typical packages must be installed before running the code:** 
> 1. datetime
> 1. plotly
> 1. chart_studio
> 1. requests

Imports of packages:

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import date, timedelta
from plotly.subplots import make_subplots
from unittest import skip
import chart_studio
import chart_studio.plotly as py


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import pandas as pd
import requests

Import of datasets:

In [2]:
# We collect data from the URL to our repository:
url_historik = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/Kurshistorik%2C%20safemax.xlsx?raw=true'
response_historik = requests.get(url_historik)
content_historik = response_historik.content
# Creating dataframe from excel file that contains the historical data
DataSafeMax = pd.read_excel(content_historik, sheet_name='KursHistorisk', usecols="B:G", skiprows=2, engine='openpyxl')
DataSafeMax = DataSafeMax.rename(columns={'DMax': 'Dmax'})
# Creating dataframe from excel file that contains the historical data
Kurserhistorik = pd.read_excel(content_historik, sheet_name='KurserHistorisk', engine='openpyxl')
Kurserhistorik.drop(Kurserhistorik.columns[0], axis=1, inplace=True)
# Now we want to create a new dataframe that contains the historical data for the funds, why we import a nwe excel file
url_indeks = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/IndeksData_udenmakro.xlsx'
response_indeks = requests.get(url_indeks)
content_indeks = response_indeks.content

url_indeks = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/IndeksData_udenmakro.xlsx?raw=true'
response_indeks = requests.get(url_indeks)
content_indeks = response_indeks.content
# The dataframe is now created
IndeksData = pd.read_excel(content_indeks, sheet_name='Fonde', usecols="C,E, G:H, AB:AC, HM:HN, HQ:HR", skiprows=1, engine='openpyxl')


BadZipFile: File is not a zip file

In [ ]:
# Now we remove data from the historical data that is already included in the SafeMax data
Kurserhistorik = Kurserhistorik[~Kurserhistorik.Dato.isin(DataSafeMax.Dato)]
Kurserhistorik = Kurserhistorik[Kurserhistorik['Dato'] < '2021-12-31']

In [ ]:
# Now we merge the two dataframes
dato_data = pd.DataFrame(pd.date_range(start='2012-10-01', end='today'), columns=['Dato'])

FinalData = pd.merge(Kurserhistorik, DataSafeMax, how='outer', sort=True).fillna(method="ffill")
FinalData = pd.merge(FinalData, dato_data, how='outer', on='Dato', sort=True).fillna(method="ffill")

In [ ]:
# Now we want to create a new dataframe that only contains the Nav data for the fund Artha Optimum, and remove the cells that contain NANs.
Optimum = FinalData.drop(columns=['Safe', 'Max', 'Responsible', 'Dmax']).dropna()

# Removing the beginning of data, because it contains the same data.
Optimum = Optimum.iloc[6:]
Today = date.today()

In [ ]:
# Removing the beginning of data, because it contains the same data.
Optimum = Optimum.iloc[6:]
Today = date.today()

In [ ]:
# Removing blanks
FinalData = FinalData[FinalData['Dato'] < datetime.datetime.now()]

# Creating list with first date of the year which we will use for rebalancing the portfolio
FirstDateOfYear = ['2013-12-30', '2014-12-30', '2015-12-30', '2016-12-30',
               '2017-12-30', '2018-12-30', '2019-12-30', '2020-12-30',
               '2021-12-30', '2022-12-30', '2023-12-30', '2024-12-30',
               '2025-12-30', '2026-12-30', '2027-12-30', '2028-12-30']
FirstDateOfYear = pd.to_datetime(FirstDateOfYear)

In [ ]:
# Creating Benchmarks dataframe consisting of mortgage bond benchmark, global stock benchmark, and risk free rate (CIBOR). 
NykreditData = IndeksData.iloc[:, 0:2].dropna()
MSCIWorldData = IndeksData.iloc[:, 2:4].dropna()
CIBOR = IndeksData.iloc[:, 4:6].dropna()

# Rename Columns
CIBOR = CIBOR.rename(columns = {CIBOR.columns[0]: 'Dato', CIBOR.columns[1]: 'CIBOR'})
NykreditData = NykreditData.rename(columns = {NykreditData.columns[0]: 'Dato', NykreditData.columns[1]: 'Nykredit'})
MSCIWorldData = MSCIWorldData.rename(columns = {MSCIWorldData.columns[0]: 'Dato', MSCIWorldData.columns[1]: 'MSCIWorld'})

# We are merging Benchmarks dataframes together 
Benchmarks = pd.merge(Optimum, pd.merge(pd.merge(MSCIWorldData, CIBOR, how='outer', on='Dato').fillna(method='ffill'), NykreditData, how='outer', on='Dato').fillna(method='ffill'), how='left', on='Dato').fillna(method='ffill')

# Calculating the daily returns
Benchmarks["Optimum daily return"]=Benchmarks["Optimum"].pct_change(1)
Benchmarks["Nykredit daily return"]=Benchmarks["Nykredit"].pct_change(1)
Benchmarks["CIBOR daily return"]=Benchmarks["CIBOR"].pct_change(1)
Benchmarks["MSCIWorld daily return"]=Benchmarks["MSCIWorld"].pct_change(1)

# Calculating the standard deviations and cumulative percentage changes (index)
Benchmarks["NykreditIndeks"] = (Benchmarks["Nykredit"]/Benchmarks["Nykredit"].iloc[0])*100
Benchmarks["OptimumIndeks"] = (Benchmarks["Optimum"]/Benchmarks["Optimum"].iloc[0])*100
Benchmarks["Optimum Std"]=Benchmarks["Optimum daily return"].rolling(365).std()*(365**0.5)
Benchmarks["Nykredit Std"]=Benchmarks["Nykredit daily return"].rolling(365).std()*(365**0.5)
Benchmarks["MSCIWorldIndeks"] = (Benchmarks["MSCIWorld"]/Benchmarks["MSCIWorld"].iloc[0])*100
Benchmarks["MSCI World Std"]=Benchmarks["MSCIWorld daily return"].rolling(365).std()*(365**0.5)
Benchmarks["CIBORIndeks"] = (Benchmarks["CIBOR"]/Benchmarks["CIBOR"].iloc[0])*100
Benchmarks["CIBOR Std"]=Benchmarks["CIBOR daily return"].rolling(365).std()*(365**0.5)

Currently our data contains daily returns, standard deviations, and inndexs for Artha Optimum and the two Benchamrks individually.
We want to end this project with som sort of visualization of the returns and risk levels of the 50/50 portfolio vs Arhta Optimum.
We wish for this visualization to be of monthly data.
To do end up there, we create our 50/50 portfolio using the daily return data, and we rebalance this portfolio ultimo each year throughout the timeframe.
(as price levels change, the weightings change, so we rebalance this at the beginning of each year.)

In [ ]:
# Now we want to compare the 50/50 portfolio with Optimum
# We only want to look at data from 2010
Optimum = Optimum[Optimum['Dato'] >= '2010-09-01']

FiftyFifty = Benchmarks.copy()
FiftyFifty = FiftyFifty[FiftyFifty['Dato'] >= '2010-09-30'].reset_index(drop=True)

# Rebalancing the  50-50 portfolio and calculating: 
# The portfolio index - used for portfolio returns. Aswell as the portoflio weights and changes in the weights - used for rebalancing.
for ind, row in FiftyFifty.iterrows():
    if FiftyFifty.loc[ind, "Dato"] in FirstDateOfYear : # if we are at the beginning of the year we rebalance
        FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = 50 * (1 + FiftyFifty.loc[ind, "Nykredit daily return"])  
        FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = 50 * (1 + FiftyFifty.loc[ind, "MSCIWorld daily return"])
        FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
        FiftyFifty.loc[ind, "Ændring vægtsum"] = (FiftyFifty.loc[ind, "Vægtsum"]/100)-1
        FiftyFifty.loc[ind, "Indeks 100"] = FiftyFifty.loc[ind - 1, "Indeks 100"] * (1 + FiftyFifty.loc[ind, "Ændring vægtsum"])
    else :
        if ind == 0 : # if we are at the first row of data we want the weights to be 50, and the index to be 100
            FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = 50
            FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = 50 
            FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
            FiftyFifty.loc[ind, "Ændring vægtsum"] = np.NaN
            FiftyFifty.loc[ind, "Indeks 100"] = 100
        else : # for everything else we just want to calculate the weights and index
            FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = FiftyFifty.loc[ind-1, "Løbende vægt Nykredit"] * (1 + FiftyFifty.loc[ind, "Nykredit daily return"])
            FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = FiftyFifty.loc[ind-1, "Løbende vægt MSCI World"] * (1 + FiftyFifty.loc[ind, "MSCIWorld daily return"])
            FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
            FiftyFifty.loc[ind, "Ændring vægtsum"] = (FiftyFifty.loc[ind, "Vægtsum"]/FiftyFifty.loc[ind-1, "Vægtsum"])-1
            FiftyFifty.loc[ind, "Indeks 100"] = FiftyFifty.loc[ind - 1, "Indeks 100"] * (1+ FiftyFifty.loc[ind, "Ændring vægtsum"])

# Remove irrelevant dates
FiftyFifty = FiftyFifty[FiftyFifty['Dato'] <= MSCIWorldData["Dato"].iloc[-1]]

# Calculating Index 
FiftyFifty["OptimumIndeks"] = (FiftyFifty["Optimum"]/FiftyFifty["Optimum"].iloc[0])*100

# Calculating Std of Optimum and Benchmark Portfolio
FiftyFifty["Optimum Std"]=FiftyFifty["Optimum daily return"].rolling(365).std()*(365**0.5)
FiftyFifty["Benchmark Std"]=FiftyFifty["Ændring vægtsum"].rolling(365).std()*(365**0.5)

In [ ]:
# Defining month
FiftyFifty['Maned'] = FiftyFifty["Dato"].dt.to_period('M')

# loop for making sure that the proper data is collected - not relevant for static dataset here, but still worth showing.
current_month = datetime.datetime.now().strftime("%B")
if current_month == 'January':
    month_value = -3
else:
    month_value = -2

# Create a dataframe containing only data from the last trading day of the year.
FiftyFiftyFirst_Dec_30 = FiftyFifty[((FiftyFifty["Dato"].dt.month == 12) & (FiftyFifty["Dato"].dt.day == 30))]
FiftyFiftyFirst = FiftyFifty.copy()
# Removing duplicates and filtering out all but the last ocurrance of each month.
FiftyFiftyFirst = FiftyFiftyFirst.drop_duplicates(subset=["Maned"], keep='last', inplace=False, ignore_index=False)

# Calculate monthly returns for Optimum and Benchmark 
FiftyFiftyFirst["Optimum maned afkast"]= FiftyFiftyFirst["Optimum"].pct_change(1)
FiftyFiftyFirst["Benchmark maned afkast"]= FiftyFiftyFirst["Indeks 100"].pct_change(1)

# Calculating Monthly CIBOR - risk free rate
FiftyFiftyFirst["CIBOR"]=(FiftyFiftyFirst["CIBOR"]/100)/12

# Calculating the excess returns which will be used for the sharpe ratio
FiftyFiftyFirst["Benchmark mer maned afkast"]=FiftyFiftyFirst["Benchmark maned afkast"]-FiftyFiftyFirst["CIBOR"]
FiftyFiftyFirst["Optimum mer maned afkast"]=FiftyFiftyFirst["Optimum maned afkast"]-FiftyFiftyFirst["CIBOR"]

# First observation equals the pre price development 
FiftyFiftyFirst.iloc[0, FiftyFiftyFirst.columns.get_loc('Optimum mer maned afkast')] = -0.006226/100
FiftyFiftyFirst.iloc[0, FiftyFiftyFirst.columns.get_loc('Benchmark mer maned afkast')] = -0.65/100

# Removing first and last rows
FiftyFiftyFirst = FiftyFiftyFirst[:-1]
FiftyFiftyFirst = FiftyFiftyFirst[1:]

# Calculating the average excess monthly return for Benchmark and Optimum
MiddelAfkastBM5050=FiftyFiftyFirst["Benchmark mer maned afkast"].mean()
MiddelAfkastOptimum5050=FiftyFiftyFirst["Optimum mer maned afkast"].mean()

# Calculating the Std for the excess return of Benchmark and Optimum
StdAfkastOptimum5050=FiftyFiftyFirst["Optimum mer maned afkast"].std()*(12**0.5)
StdAfkastBM5050 = FiftyFiftyFirst["Benchmark mer maned afkast"].std()*(12**0.5)

# Calculating the Sharpe Ratio for the Benchmark and Optimum
SharpeOptimum5050=(MiddelAfkastOptimum5050/FiftyFiftyFirst["Optimum mer maned afkast"].std())*(12**0.5)
SharpeBM5050=(MiddelAfkastBM5050/FiftyFiftyFirst["Benchmark mer maned afkast"].std())*(12**0.5)

# Making use of the ultimo year dataframe for the YTD calculations
FiftyFiftyFirst = pd.concat([FiftyFiftyFirst_Dec_30, FiftyFiftyFirst], axis=0).sort_values('Dato').reset_index(drop=True)

# Date column is set to last trading day of the year, of the previous year 
FiftyFiftyFirstATD = FiftyFifty.resample('Y', on='Dato')['Dato'].agg(['last']).iat[month_value, 0] - timedelta(days=1)
OptimumFirstATD = FiftyFifty[FiftyFifty['Dato'] == FiftyFiftyFirstATD]
# Same as above - except that the final day is returned
FiftyFiftyFirstATD_31 = FiftyFifty.resample('Y', on='Dato')['Dato'].agg(['last']).iat[month_value, 0]
OptimumFirstATD_31 = FiftyFifty[FiftyFifty['Dato'] == FiftyFiftyFirstATD_31]

# Calculating returns YTD
AfkastATD5050Optimum = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Optimum')]/OptimumFirstATD_31.iloc[0, OptimumFirstATD_31.columns.get_loc('Optimum')])-1
AfkastATD5050BM = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Indeks 100')]/OptimumFirstATD.iloc[0, OptimumFirstATD.columns.get_loc('Indeks 100')])-1

# Calculating the return for the whole period  
AfkastTotal5050Optimum = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Optimum')]/FiftyFifty.iloc[0, FiftyFifty.columns.get_loc('Optimum')])-1
AfkastTotal5050BM = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Indeks 100')]/100)-1




We have now made all the necessary calculations. Let us plot the results.

In [ ]:
# We create a function for plotting two graphs: 1. containing the returns comparison and 2. containing the risk comparison

def kurserplot(FigurNavn, DatoSoejle, ArthaFond, FondsNavn, BMFond, BMNavn, StdFond, StdBM):
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=DatoSoejle, y=ArthaFond, mode='lines', line_color='#AD9E74', line_width=1.5, name=FondsNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig.add_trace(go.Scatter(x=DatoSoejle, y=BMFond, mode='lines', line_color='#131D2F', line_width=1.5, name=BMNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig.update_xaxes(title_text='', showgrid=False)

    ymax = max([ArthaFond.max(), BMFond.max()]) + 10
    ymin = min([ArthaFond.min(), BMFond.min()]) - 10

    fig.update_yaxes(title_text='', range=[ymin, ymax], gridcolor="#F0F0F0")

    fig.update_layout(
    plot_bgcolor="white",
    font_color="black",
    title=dict(text = "Return period: " + DatoSoejle.iloc[365].strftime("%d-%m-%Y") + " til " + DatoSoejle.iloc[-1].strftime("%d-%m-%Y"), font=dict(size=16)),
    xaxis_range=[DatoSoejle.iloc[0], DatoSoejle.iloc[-1]],
    margin=dict(t=30,l=0,b=10,r=10),
    width=600,
    height=450,
    #xaxis=dict(tickformat="%Y"),
    font_family="Grandview",
    font = dict(
        size = 16
    ),
    legend=dict(
    orientation="h"
    )
    )
    fig.show()
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=DatoSoejle, y=StdFond, mode='lines', line_color='#AD9E74',
                     line_width=1.5, name=FondsNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig2.add_trace(go.Scatter(x=DatoSoejle, y=StdBM, mode='lines', line_color='#131D2F',
                     line_width=1.5, name=BMNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig2.update_xaxes(title_text='', showgrid=False)
    fig2.update_yaxes(title_text='', gridcolor="#F0F0F0")
    fig2.update_layout(
        plot_bgcolor="white",
        font_color="black",
        
        title=dict(text = "Risiko periode: " + DatoSoejle.iloc[365].strftime("%d-%m-%Y") + " til " + DatoSoejle.iloc[-1].strftime("%d-%m-%Y"), font=dict(size=16)),
        xaxis_range=[DatoSoejle.iloc[365], DatoSoejle.iloc[-1]],
        yaxis_tickformat = '.0%',margin=dict(t=30,l=0,b=10,r=10),
        
        width=600,
        height=450,
        font_family="Grandview",
        font = dict(
            size = 16
        ),
        legend=dict(
        orientation="h"
    )
    )
    fig2.show()

now = datetime.datetime.now().date()
eop = now.replace(day=1)
eop = str(eop)

FiftyFifty = FiftyFifty[FiftyFifty['Dato'] < eop]


kurserplot("OptimumGraf", FiftyFifty['Dato'], FiftyFifty['OptimumIndeks'], "Optimum", FiftyFifty['Indeks 100'], "50/50% mortg.bonds/global stocks", FiftyFifty['Optimum Std'], FiftyFifty['Benchmark Std'])


Atha Optimum outperforms the Benchmark portfolio historically wrt. both returns and risk. Nice!
For comparisons like this it can also be nice to not only see historical numbers, but also current numbers as returns YTD.
We therefore create a table containing risk and returns historically, returns YTD and our sharpe ratio calculations.
The Sharpe ratio numbers will round off the whole comparison, as they give us an idea of how each portfolio performs wrt. both risk and returns (and not just returns, but excess returns above the risk free rate - CIBOR)

In [ ]:
# Defining colors
colors = ['white', 'white', '#F0F0F0']
data = {'Color' : colors}
df = pd.DataFrame(data)

# Nested list containing five sublists wehich contain rows and columns of the table
ValuesOpti=[["Optimum", "50/50% mortg.bonds/global stocks", "<b>Difference</b>"],
        ["{:.1%}".format(round(AfkastTotal5050Optimum,3)),"{:.1%}".format(round(AfkastTotal5050BM,3)), "{:.1%}".format(round(AfkastTotal5050Optimum-AfkastTotal5050BM,3))],
        ["{:.1%}".format(round(AfkastATD5050Optimum,3)), "{:.1%}".format(round(AfkastATD5050BM,3)),  "{:.1%}".format(round(AfkastATD5050Optimum-AfkastATD5050BM,3))],
        ["{:.1%}".format(round(StdAfkastOptimum5050,3)), "{:.1%}".format(round(StdAfkastBM5050,3)),  "{:.1%}".format(round(StdAfkastOptimum5050-StdAfkastBM5050,3))],
        ["{:.2f}".format(round(SharpeOptimum5050, 2)), "{:.2f}".format(round(SharpeBM5050, 2)), "{:.2f}".format(round(SharpeOptimum5050-SharpeBM5050, 2))]]

# Function for  building the actual table using go.Table
def Table(FigurNavn, Values):
    Header=["<b>Return for the period</b>", "<b>Return since start</b>", "<b>Return YTD</b>", "<b>Risk</b>", "<b>Sharpe</b>"]
    TableFig = go.Figure(data=[go.Table(
        columnwidth = [310,160,120,80,80],
        header=dict(
        values=Header, 
        align=['left','center'],
        fill_color="white",
        font=dict(color='black', size=16)
        ),
        cells=dict(values=Values, 
        align=['left','center'],
        height=30,
        line_color=[df.Color],
        fill_color=[df.Color],
        font=dict(color='black', size=16)))])
    TableFig.show()
    TableFig.update_layout(width=700, height=150, margin={"l":0,"r":0,"t":0,"b":0}, font_family="Grandview")


Table("OptimumTest", ValuesOpti)


# Conclusion
Looking at the Difference row in the table above we clearly see that Artha Optimum outperrforms the Benchmark on all metrics. All in all we can conlude that Artha optimum both historically and presntly creates a higher return at a lower risk level, which results in a much higher sharpe ratio.